# Smart Dough 1.0

This is a tool for doing all the necessary calculations before you make bread. It starts off by asking you to make some decisions about the dough - things such as how much bread you want to make, what hydration you want, how much wholewheat you want to use, etc.

After this initial round of questions, you will also be asked some questions about your schedule. You'll be asked things like when you want to start, how long you want to autolyse, how long to bulk ferment, etc.

Finally, after all this, the app will output a schedule you can use, that details exactly how much of each ingredient needs to be added at each step. At the end, it will also give you a summary of the dough specs.

So, once you're ready to go, answer the questions below and get baking!

## Assumptions

Note that this app makes several assumptions, listed below:
- your bread will only contain white and wholewheat flour, never any other kind of flour.
- your levain will also only contain white and wholewheat flour, never any other kind of flour.
- your levain is 100% hydration, and you will always build your levain at 1\:2\:2 (starter\:flour\:water).
- the workflow will always follow the format of an autolyse, followed by a warm bulk ferment, and finally a long final proof (retard) in the fridge.

## Definitions

There are a lot of terms in bread baking that mean different things depending on who you ask. Below are my definitions for all the terms used in the app:
- **flour mix (and percentage wholewheat):** for any amount of dough, the total amount of flour in the dough (including all flour types, as well as flour from the levain) is taken to equal 100%. When asking for the percentage of wholewheat, this corresponds to the proprortion of the *total flour* that is wholewheat. So say you choose 20%, that means that 20% of all flour in the dough will be wholewheat. Note that the app takes into account the percentage wholewheat in your levain and makes the necessary adjustments.
- there are lots more, come back and finish this section at some stage.

## Ideas for improving the app
- Finish the definitions section.
- Give an error message for the questions if an answer is out of range.
- Add the ability to save the final output as a file.
- Specify the number of flours used, for the dough as well as the starter.
- Customise the ratio with which you build your levain.
- Automatically detect when activities run overnight and add a line then, instead of hardcoding them in.
- Change the input to a table format, rather than individually prompted questions.

In [1]:
# Some questions about the dough
num_loaves = float(input("How many loaves would you like to make?"))
size_loaves = float(input("How big would you like the loaves to be (in grams)?"))
pct_wholewheat = float(input("What percentage of wholewheat would you like to use (the rest is assumed to be white flour)?"))
hydration = float(input("What hydration do you want your dough to be (water as a percentage of total flour)?"))
levain = float(input("How much levain do you want to use (as a percentage of total flour)?"))
hydration_levain = float(input("What hydration is your levain (water as a percentage of flour)?"))
pct_ww_levain = float(input("What percentage wholewheat is your levain?"))
salt = float(input("How much salt do you want to use (as a percentage of total flour)?"))

How many loaves would you like to make? 2
How big would you like the loaves to be (in grams)? 700
What percentage of wholewheat would you like to use (the rest is assumed to be white flour)? 18
What hydration do you want your dough to be (water as a percentage of total flour)? 78
How much levain do you want to use (as a percentage of total flour)? 20
What hydration is your levain (water as a percentage of flour)? 100
What percentage wholewheat is your levain? 50
How much salt do you want to use (as a percentage of total flour)? 2.4


In [2]:
import datetime as dt

# Some questions about the schedule
start_time = dt.datetime.strptime(input("What time of day do you want to start at (in the form HH:MM)?"), "%H:%M")
levain_time = float(input("How long do you want to give your levain before you add it to the dough (in hours)?"))
autolyse_time = float(input("How long do you want the autolyse to be (in hours)?"))
post_levain_time = float(input("How long do you want to leave between adding the levain and adding the salt (in minutes)?"))
post_salt_time = float(input("How long do you want to leave between adding the salt and doing the lamination (in minutes)?"))
bulk_time = float(input("How long do you want the bulk fermentation to be (in hours)?"))
bench_rest = float(input("How long do you want to bench rest between pre-shape and final shaping (in minutes)?"))
retard_time = float(input("How long do you want to retard the dough in the fridge after shaping (in hours?)"))

What time of day do you want to start at (in the form HH:MM)? 09:30
How long do you want to give your levain before you add it to the dough (in hours)? 6
How long do you want the autolyse to be (in hours)? 1.5
How long do you want to leave between adding the levain and adding the salt (in minutes)? 30
How long do you want to leave between adding the salt and doing the lamination (in minutes)? 30
How long do you want the bulk fermentation to be (in hours)? 4.5
How long do you want to bench rest between pre-shape and final shaping (in minutes)? 30
How long do you want to retard the dough in the fridge after shaping (in hours?) 14


In [3]:
# Calculate weight of total amount of dough
total_dough = (num_loaves * size_loaves) * 1.03 # add 3% to account for loss during mixing and transfer

# Calculate amount of flour in the dough
total_pct = 100 + hydration + salt
total_flour = (total_dough / total_pct) * 100

# Calculate the amount of salt
total_salt = total_flour * (salt / 100)

# Calculate the amount of levain
total_levain = total_flour * (levain / 100)
flour_from_levain = total_levain * (100 / (100 + hydration_levain))
white_from_levain = flour_from_levain * ((100 - pct_ww_levain) / 100)
ww_from_levain = flour_from_levain * pct_ww_levain / 100
water_from_levain = total_levain * (hydration_levain / (100 + hydration_levain))

# Calculate levain mix quantities (assumes a 1:2:2 mix)
prep_levain = total_levain * 1.1 # add 10% to account for some loss of mass during fermentation and transfer
prep_starter = prep_levain / 5
prep_water = prep_starter * 2
prep_flour = prep_starter * 2

# Calculate the amount of water
total_water = total_flour * (hydration / 100)
autolyse_water = total_water - water_from_levain

# Calculate the amount of flour
white_flour = total_flour * ((100 - pct_wholewheat) / 100)
wholewheat_flour = total_flour * (pct_wholewheat / 100)
autolyse_white = white_flour - white_from_levain
autolyse_ww = wholewheat_flour - ww_from_levain

# Convert all measures to integers for printing later
prep_starter = round(prep_starter)
prep_flour = round(prep_flour)
prep_water = round(prep_water)
prep_levain = round(prep_levain)
total_levain = round(total_levain)
autolyse_white = round(autolyse_white)
autolyse_ww = round(autolyse_ww)
autolyse_water = round(autolyse_water)
total_salt = round(total_salt)
num_loaves = round(num_loaves)
size_loaves = round(size_loaves)
total_dough = round(total_dough)
pct_wholewheat = round(pct_wholewheat)
hydration = round(hydration)
levain = round(levain)

In [4]:
# Calculate when levain gets added
levain_mix = start_time + dt.timedelta(hours = levain_time)

# Calculate when which autolyse is mixed
autolyse_mix = levain_mix - dt.timedelta(hours = autolyse_time)

# Calculate when salt gets added
salt_mix = levain_mix + dt.timedelta(minutes = post_levain_time)

# Calculate when pre-shaping is done
pre_shape = salt_mix + dt.timedelta(hours = bulk_time)

# Calculate when final shaping and retarding happens
shaping = pre_shape + dt.timedelta(minutes = bench_rest)

# Calculate when the loaves are baked
baking = shaping + dt.timedelta(hours = retard_time)

# Calculate when the oven needs to be pre-heated
oven = baking - dt.timedelta(minutes = 30)

# Convert all times to strings in the format HH:MM for printing
start_time = str(start_time.time())[:5]
autolyse_mix = str(autolyse_mix.time())[:5]
levain_mix = str(levain_mix.time())[:5]
salt_mix = str(salt_mix.time())[:5]
lamination = str(lamination.time())[:5]
pre_shape = str(pre_shape.time())[:5]
shaping = str(shaping.time())[:5]
baking = str(baking.time())[:5]
oven = str(oven.time())[:5]
levain_time = round(levain_time)
autolyse_time = round(autolyse_time)
bench_rest = round(bench_rest)
retard_time = round(retard_time)

In [9]:
# Create the schedule
line_width = 7
print("\033[1m" + start_time.ljust(line_width), "Prepare the levain" + "\033[0m")
print("".ljust(line_width), "* Combine {}g mature starter, {}g of your starter flour mix and {}g water.".format(prep_starter, prep_flour, prep_water))
print("".ljust(line_width), "* This will give you {}g of levain. You'll only need {}g to add to the dough later.".format(prep_levain, total_levain))
print("".ljust(line_width), "* Try keep your levain around 25\u00b0C, and keep an eye on it.")
print("")
print("-"*123)
print("")
print("\033[1m" + autolyse_mix.ljust(line_width), "Autolyse" + "\033[0m")
print("".ljust(line_width), "* In a large bowl, combine {}g white flour, {}g wholewheat flour and {}g water".format(autolyse_white, autolyse_ww, autolyse_water))
print("".ljust(line_width), "* Heat the water to around 30-40\u00b0C (you're aiming for a final dough temperature of 25\u00b0C).")
print("".ljust(line_width), "* Mix until just combined, and there are no bits of dry flour left.")
print("".ljust(line_width), "* Cover and leave in the same place as your levain.")
print("")
print("\033[1m" + levain_mix.ljust(line_width), "Levain mix" + "\033[0m")
print("".ljust(line_width), "* Pour {}g levain on top of the dough.".format(total_levain))
print("".ljust(line_width), "* Using a moistened hand, mix the dough for 4 minutes using the pinch and fold technique.")
print("".ljust(line_width), "* Cover and place back in your warm spot.")
print("")
print("\033[1m" + salt_mix.ljust(line_width), "Salt mix" + "\033[0m")
print("".ljust(line_width), "* Sprinkle {}g salt on top of the dough.".format(total_salt))
print("".ljust(line_width), "* Using a moistened hand, mix the dough for 4 minutes using the pinch and fold technique.")
print("".ljust(line_width), "* Cover and place back in your warm spot.")
print("")
print("\033[1m" + salt_mix.ljust(line_width), "Bulk fermentation" + "\033[0m")
print("\033[1m" + "  - " + "\033[0m".ljust(line_width), "* During this time, perform as many folds as is necessary to build a strong dough (usually 2-4 folds).")
print("\033[1m" + pre_shape.ljust(line_width) + "\033[0m", "* Space the folds out so the dough is left untouched for the last 2 hours or so of bulk.")
print("")
print("\033[1m" + pre_shape.ljust(line_width), "Pre-shape" + "\033[0m")
print("".ljust(line_width), "* Transfer the dough to an unfloured work surface.")
if num_loaves > 1:
    print("".ljust(line_width), "* Using a moistened bench scraper, divide the dough into {} {}g loaves.".format(num_loaves, size_loaves))
    print("".ljust(line_width), "* For each loaf, shape into a tight boule with your bench scraper.")
else:
    print("".ljust(line_width), "* Perform a final coild fold on the bench, and shape into a tight boule with your bench scraper.")
print("")
print("\033[1m" + shaping.ljust(line_width), "Shaping" + "\033[0m")
print("".ljust(line_width), "* Shape into a batard or a boule, depending on what you have.")
print("".ljust(line_width), "* After you have shaped a loaf, leave it to rest seam side down on the counter for 5 minutes.")
print("".ljust(line_width), "* Transfer to a lined banneton, cover, and place in the fridge.")
print("")
print("-"*123)
print("")
print("\033[1m" + oven.ljust(line_width), "Pre-heat oven" + "\033[0m")
print("".ljust(line_width), "* Turn the oven on to 240\u00b0C, with your baking dish in there to heat up as well.")
print("")
print("\033[1m" + baking.ljust(line_width), "Bake" + "\033[0m")
print("".ljust(line_width), "* Remove loaf from the fridge and turn out on to parchment paper.")
print("".ljust(line_width), "* Score, and transfer into pre-heated baking dish.")
print("".ljust(line_width), "* Add extra steam if necessary, and place in oven.")
print("".ljust(line_width), "* Bake for 20-30 minutes, then vent steam by opening oven door and removing the lid of the baking dish.")
print("".ljust(line_width), "* Bake for another 15-25 minutes, until the crust is well developed.")
if num_loaves > 1:
    print("".ljust(line_width), "* Make sure to re-heat the baking dish for 10-15 minutes between loaves.")
print("".ljust(line_width), "* Transfer to a wire rack and leave to cool completely.")
print("")
print("-"*123)
print("")
print("\033[1m" + "Final dough specs:" + "\033[0m")
if num_loaves > 1:
    print("* {} x {}g loaves ({}g of dough total).".format(num_loaves, size_loaves, total_dough))
else:
    print("* {} x {}g loaf ({}g of dough total).".format(num_loaves, size_loaves, total_dough))
print("* {}% white flour, {}% wholewheat flour.".format(100-pct_wholewheat, pct_wholewheat))
print("* {}% hydration.".format(hydration))
print("* {}% levain.".format(levain))
print("* {}% salt.".format(salt))
print("")
print("\033[1m" + "Method overview:" + "\033[0m")
print("* Levain was built at 1:2:2 and developed for {} hours.".format(levain_time))
print("* The autolyse was {} hours long.".format(autolyse_time))
print("* Bulk fermenation lasted {} hours.".format(bulk_time))
print("* {} minutes bench rest after shaping.".format(bench_rest))
print("* Retarded for {} hours.".format(retard_time))

09:30   Prepare the levain
        * Combine 35g mature starter, 70g of your starter flour mix and 70g water.
        * This will give you 176g of levain. You'll only need 160g to add to the dough later.
        * Try keep your levain around 25°C, and keep an eye on it.

---------------------------------------------------------------------------------------------------------------------------

14:00   Autolyse
        * In a large bowl, combine 615g white flour, 104g wholewheat flour and 544g water
        * Heat the water to around 30-40°C (you're aiming for a final dough temperature of 25°C).
        * Mix until just combined, and there are no bits of dry flour left.
        * Cover and leave in the same place as your levain.

15:30   Levain mix
        * Pour 160g levain on top of the dough.
        * Using a moistened hand, mix the dough for 4 minutes using the pinch and fold technique.
        * Cover and place back in your warm spot.

16:00   Salt mix
        * Sprinkle 19g salt 

In [6]:
print("Flour: {}% ({}g/{}g)".format(round(((round(total_flour)/round(total_dough))*100),2), round(total_flour), round(total_dough)))
print("Water: {}% ({}g/{}g)".format(round(((round(total_water)/round(total_dough))*100),2), round(total_water), round(total_dough)))
print("Salt: {}% ({}g/{}g)".format(round(((round(total_salt)/round(total_dough))*100),2), round(total_salt), round(total_dough)))
print("Levain: {}% ({}g/{}g)".format(round(((round(total_levain)/round(total_dough))*100),2), round(total_levain), round(total_dough)))

Flour: 52.94% (954g/1802g)
Water: 46.06% (830g/1802g)
Salt: 1.05% (19g/1802g)
Levain: 8.49% (153g/1802g)
